In [1]:
!pip install opencage


from tqdm import tqdm

from opencage.geocoder import OpenCageGeocode

import os
import ast
import pandas as pd
import numpy as np

In [2]:

#TODO replace with df_meta then when join with df_pass in next notebook will make sense
# no maybe has to be like this becuase df meta does not have "filing missing" column

base = 'data/'


# load data and remove empty columns
df_speakers = pd.read_csv(base+'speakers_all.csv')
df_speakers = df_speakers.drop(columns=[ 'Unnamed: 9', 'Unnamed: 10','Unnamed: 11'])

# remove speakers that don't have audio file
df_speakers = df_speakers[~df_speakers['file_missing?']]
# only needs to be run once!!!
####### Get Lat Lng for birthplace and country of residence #########

# for GEO API
key = '5e96746aec93410f93415a7b9aee3240'
geocoder = OpenCageGeocode(key)

def get_latlng(place):
    """
    Purpose: get lat,lng of city,state, country or city, country
    """
    
    try:
        result = geocoder.geocode(place)    

        # parse API results
        lat = result[0]['annotations']['DMS']['lat']
        lng = result[0]['annotations']['DMS']['lng']
        return lat,lng
    
    except:
        # if failed try another way
        print('failed',place)
        return None
    
    
# fill in latlon for birthplace of each speaker in df
# don't relook up same place more than once because the API has a limit and I dont have all day
birthplace_lanlng_map = {birthplace:get_latlng(birthplace) for birthplace in tqdm(df_speakers['birthplace'].unique())}
df_speakers['birthplace_latlng'] = df_speakers['birthplace'].map(birthplace_lanlng_map)
print('missed' ,df_speakers['birthplace_latlng'].isna().sum() , 'birthplace lat lngs')

country_lanlng_map = {country:get_latlng(country) for country in df_speakers['country'].unique()}
df_speakers['country_latlng'] = df_speakers['country'].map(birthplace_lanlng_map)
print('missed' ,df_speakers['birthplace_latlng'].isna().sum() , 'country lat lngs')



# save it down cause who wants to do this twice
df_speakers.to_json(base+'speakers_latlng.json')


 80%|████████  | 1020/1271 [23:05<07:56,  1.90s/it]

failed babadag, romanian


 84%|████████▎ | 1062/1271 [24:02<04:38,  1.33s/it]

failed zrenjanin, serbia, yugoslavia


 93%|█████████▎| 1176/1271 [26:42<02:11,  1.38s/it]

failed nan


100%|██████████| 1271/1271 [28:50<00:00,  1.36s/it]


missed 6 birthplace lat lngs
failed nan
missed 6 country lat lngs


In [ ]:
df_speakers.head()